In [1]:
import pandas as pd
import scipy
import numpy as np
import re
import nltk
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
#ratings = pd.merge(movies, ratings).drop(['date_added', 'release_year', 'rating', 'duration'], axis=1)
ratings = pd.merge(movies, ratings)

ratings.drop('timestamp', axis = 1, inplace=True)
ratings.head()

In [ ]:
ratings['rating'] = ratings['rating'].fillna('0')
ratings.head()

In [ ]:
ratings['clean_genre'] = ratings['genres'].str.lower()
ratings['clean_genre'] = ratings['clean_genre'].apply(lambda x: re.sub(' ', '', x))
ratings['clean_genre'] = ratings['clean_genre'].apply(lambda x: re.sub('\s+', ' ', x))
ratings['clean_genre'] = ratings['clean_genre'].apply(lambda x: x.split('|'))
#pd.set_option('display.max_rows', 100)
#ratings['clean_genre']
ratings

In [ ]:
genre_ratings = ratings[['title', 'userId', 'rating', 'clean_genre']].copy()
genre_ratings.head()

In [ ]:
def get_similar_genre(genre):
    #similar_genre = genre_ratings['user_id', 'clean_genre']
    temp = []
    df = pd.DataFrame()
    #print(genre_ratings['clean_genre'][0][0])
    for i, x in enumerate(genre_ratings['clean_genre']):
        for y in x:
            if(y in genre):
                temp.append(i)
                     
    #temp = genre_ratings.loc[genre_ratings['clean_genre'][0][0] == ([genre])]
    #temp = genre_ratings.loc[genre_ratings['clean_genre'].str.contains(genre)]
    temp = set(temp)
    temp = list(temp)
    df = genre_ratings.iloc[temp]
    return df

In [ ]:
df_genre = get_similar_genre(["children"])
df_genre
#df_genre.drop_duplicates(inplace=True)
#get_similar_genre(["adventure"])

In [ ]:
recc_ratings = df_genre.pivot_table(index=['userId'],columns=['title'],values='rating')
recc_ratings.fillna(0, inplace=True)
recc_ratings

In [ ]:
recc_columns = recc_ratings.sum(axis=0).sort_values(ascending=False)
#recc_columns = recc_ratings.pivot_table(columns=[''])
recc_columns = pd.DataFrame(recc_columns)
recc_columns['sum'] = recc_ratings.sum(axis=0).sort_values(ascending=False)
#recc_columns.(columns=recc_columns[0], axis=1, inplace=True)
del recc_columns[recc_columns.columns[0]]
recc_columns

In [ ]:
recc_normalization = df_genre['title'].value_counts()
recc_normalization = pd.DataFrame(recc_normalization)
recc_normalization.rename(columns={'title':'count'}, inplace=True)
recc_normalization

In [ ]:
#recc_merge = pd.join(recc_columns, recc_normalization)
recc_merge = recc_columns.join(recc_normalization)
recc_merge

## Normalizing + removing count values <= 100

In [ ]:
recc_merge['normalize'] = (recc_merge['sum']/recc_merge['count'])
recc_merge['count'].dtypes
#recc_merge = recc_merge.drop(recc_merge[recc_merge['count'] < 100].index, inplace=True)
recc_merge = recc_merge[(recc_merge['count'] > 100)]
recc_merge.sort_values(by='count', ascending=False).head(10)

# NETFLIX DATA 

In [2]:
netflix_ratings = pd.read_csv('userDatas.csv')
netflix_movies = pd.read_csv('netflix_titles.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

#movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [3]:
movies.title = movies.title.str.replace('\d+', '')
movies.title = movies.title.str.replace('(', '')
movies.title = movies.title.str.replace(')', '')
#movies['title'] = movies['title'].apply(lambda x: re.sub(' ', '', x))
movies['title'] = movies['title'].str.lower()
movies['title'] = movies['title'].str.strip()
#movies

/var/folders/p4/0gmnlm6d5kb64rkt5b8spdzm0000gn/T/ipykernel_60392/3976701913.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movies.title = movies.title.str.replace('\d+', '')
/var/folders/p4/0gmnlm6d5kb64rkt5b8spdzm0000gn/T/ipykernel_60392/3976701913.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movies.title = movies.title.str.replace('(', '')
/var/folders/p4/0gmnlm6d5kb64rkt5b8spdzm0000gn/T/ipykernel_60392/3976701913.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movies.title = movies.title.str.replace(')', '')


In [4]:
netflix_movies['title'] = netflix_movies['title'].str.lower()
#netflix_movies['title'] = netflix_movies['title'].apply(lambda x: re.sub(' ', '', x))
#netflix_movies

In [5]:
overall_movies = movies.merge(netflix_movies, on='title', how='left')
#overall_movies = movies.join(netflix_movies)
#overall_movies = pd.concat([movies, netflix_movies], axis=1, join='inner')
#overall_movies

In [6]:
overall_movies = overall_movies.dropna()

In [7]:
overall_movies = overall_movies[['movieId', 'title', 'genres']]
overall_movies = overall_movies.merge(ratings)
overall_movies = overall_movies.drop('timestamp', axis='columns')
#overall_movies

In [8]:
overall_movies['clean_genre'] = overall_movies['genres'].str.lower()
overall_movies['clean_genre'] = overall_movies['clean_genre'].apply(lambda x: re.sub(' ', '', x))
overall_movies['clean_genre'] = overall_movies['clean_genre'].apply(lambda x: re.sub('\s+', ' ', x))
overall_movies['clean_genre'] = overall_movies['clean_genre'].apply(lambda x: x.split('|'))
#overall_movies

In [9]:
def get_similar_netflix_genre(genre):
    #similar_genre = genre_ratings['user_id', 'clean_genre']
    temp = []
    df = pd.DataFrame()
    #print(genre_ratings['clean_genre'][0][0])
    for i, x in enumerate(overall_movies['clean_genre']):
        for y in x:
            if(y in genre):
                temp.append(i)
                     
    #temp = genre_ratings.loc[genre_ratings['clean_genre'][0][0] == ([genre])]
    #temp = genre_ratings.loc[genre_ratings['clean_genre'].str.contains(genre)]
    temp = set(temp)
    temp = list(temp)
    df = overall_movies.iloc[temp]
    return df

In [10]:
df_netflix_genre = get_similar_netflix_genre(["children"])
#df_netflix_genre

In [11]:
recc_netflix_ratings = df_netflix_genre.pivot_table(index=['userId'],columns=['title'],values='rating')
recc_netflix_ratings.fillna(0, inplace=True)
#recc_netflix_ratings

In [12]:
recc_netflix_columns = recc_netflix_ratings.sum(axis=0).sort_values(ascending=False)
#recc_columns = recc_ratings.pivot_table(columns=[''])
recc_netflix_columns = pd.DataFrame(recc_netflix_columns)
recc_netflix_columns['sum'] = recc_netflix_ratings.sum(axis=0).sort_values(ascending=False)
#recc_columns.(columns=recc_columns[0], axis=1, inplace=True)
del recc_netflix_columns[recc_netflix_columns.columns[0]]
#recc_netflix_columns

In [13]:
recc_netflix_normalization = df_netflix_genre['title'].value_counts()
recc_netflix_normalization = pd.DataFrame(recc_netflix_normalization)
recc_netflix_normalization.rename(columns={'title':'count'}, inplace=True)
#recc_netflix_normalization

In [14]:
recc_netflix_merge = recc_netflix_columns.join(recc_netflix_normalization)
recc_netflix_merge

,sum,count
title,,
willy wonka & the chocolate factory,461.0,119
kung fu panda,191.0,71
charlie and the chocolate factory,183.0,61
free willy,88.5,37
tarzan,83.0,24
spy kids,82.5,28
charlotte's web,81.0,24
teenage mutant ninja turtles,75.0,25
all dogs go to heaven,70.0,26


In [15]:
recc_netflix_merge['normalize'] = (recc_netflix_merge['sum']/recc_netflix_merge['count'])
recc_netflix_merge['count'].dtypes
#recc_merge = recc_merge.drop(recc_merge[recc_merge['count'] < 100].index, inplace=True)
recc_netflix_merge = recc_netflix_merge[(recc_netflix_merge['count'] > 10)]
recc_netflix_merge.sort_values(by='count', ascending=False).head(10)

,sum,count,normalize
title,,,
willy wonka & the chocolate factory,461.0,119,3.873950
kung fu panda,191.0,71,2.690141
charlie and the chocolate factory,183.0,61,3.000000
free willy,88.5,37,2.391892
spy kids,82.5,28,2.946429
all dogs go to heaven,70.0,26,2.692308
teenage mutant ninja turtles,75.0,25,3.000000
stuart little,56.0,24,2.333333
tarzan,83.0,24,3.458333


In [ ]:
ratings['cast'] = ratings['cast'].fillna(' ')
ratings['fav_cast'] = ratings['fav_cast'].fillna(' ')
ratings['director'] = ratings['director'].fillna(' ')
ratings['country'] = ratings['country'].fillna(' ')

ratings.head(61)

In [ ]:
userRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='user_rating')
print("Before: ",userRatings.shape)
#userRatings = userRatings.dropna(how='all').fillna(0)
userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)
userRatings.head(10)

In [ ]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(10)

In [ ]:
def standardize(row):
    new_row = (row - row.mean())/(row.max()-row.min())
    return new_row

df_temp = userRatings
df_std = df_temp.apply(standardize)

user_similarity = cosine_similarity(df_std.T)
user_similarity

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
euclidean_dist = euclidean_distances(df_std.T)
euclidean_dist

In [ ]:
user_similarity_df = pd.DataFrame(user_similarity,index=userRatings.columns,columns=userRatings.columns)
user_similarity_df

In [ ]:
def get_similar_cosine(movie_name, user_rating):
    similar_score = user_similarity_df[movie_name]
    similar_score = similar_score.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_score

In [ ]:
new_user1 = [("ONE PIECE",5),("Sword Art Online",2),("Sex Education",4),("Hunter X Hunter (2011)",5),("Attack on Titan",4)]
similar_movies = pd.DataFrame()
for movie,rating in new_user1:
    similar_movies = similar_movies.append(get_similar_cosine(movie,rating),ignore_index = True)

for (x,_) in new_user1:
    del similar_movies[x]
    #print("drop "+ x)
similar_movies.head()

In [ ]:
similar_movies.sum().sort_values(ascending=False).head(10)

In [ ]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(10)

In [ ]:
def get_similar_pearson(movie_name, user_rating):
    similar_score = corrMatrix[movie_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_score

In [ ]:
new_user1 = [("ONE PIECE",5),("Sword Art Online",2),("Sex Education",4),("Hunter X Hunter (2011)",5),("Attack on Titan",4)]
similar_movies = pd.DataFrame()
for movie,rating in new_user1:
    similar_movies = similar_movies.append(get_similar_pearson(movie,rating),ignore_index = True)

for (x,_) in new_user1:
    del similar_movies[x]
    #print("drop "+ x)
similar_movies.head()

In [ ]:
similar_movies.sum().sort_values(ascending=False).head(10)

In [ ]:
action_lover = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
similar_movies = pd.DataFrame()
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar_pearson(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)

#### Evalaution Metric : RMSE ( Root Mean Squared Error )

In [ ]:
# Function to predict ratings
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    return sqrt(mean_squared_error(pred, actual))

# Predict ratings on the training data with both similarity score
user_prediction = predict(euclidean_dist, corrMatrix, type='user')
item_prediction = predict(euclidean_dist, corrMatrix, type='item')
# RMSE on the train data
print('User-based CF RMSE: ' + str(rmse(user_prediction, corrMatrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, corrMatrix)))